In [33]:
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier

In [34]:
def residual(y, z):
    return y - z

In [94]:
def gb_predict(X, trees_list, eta):
    # Реализуемый алгоритм градиентного бустинга будет инициализироваться нулевыми значениями,
    # поэтому все деревья из списка trees_list уже являются дополнительными и при предсказании
    # прибавляются с шагом eta

    if trees_list:
        predictions = sum([eta * np.array(predict(X, alg)) for alg in trees_list])
    else:
        predictions = 0

    return predictions

In [95]:
def gb_fit(n_trees, max_depth, X_train, y_train, eta, positive_class=1, min_samples_leaf=5):
    
    # Деревья будем записывать в список
    trees = []
    initial_prediction = np.array([np.log(y_train[y_train == positive_class].shape[0] / 
                               y_train[y_train != positive_class].shape[0])] * y_train.shape[0])
    
    for i in range(n_trees):

        target = initial_prediction + gb_predict(X_train, trees, eta)
        probs = np.exp(target) / (1 + np.exp(target))
        tree = build_tree(X_train, residual(y_train, probs), probs, max_depth, min_samples_leaf)
        

        trees.append(tree)
        
    return trees

In [76]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле



# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, values, probs):
        self.data = data
        self.values = values
        self.probs = probs
        self.prediction = self.predict()
        
    def predict(self):
        # Рассчитываем гамму
        return np.sum(self.values) / np.sum([p*(1-p) for p in self.probs])


# Расчет прироста

def gain(left_values, right_values, root_var):

    # доля выборки, ушедшая в левое поддерево
    p = float(left_values.shape[0]) / (left_values.shape[0] + right_values.shape[0])
    
    return root_var - p * np.var(left_values) - (1 - p) * np.var(right_values)


# Разбиение датасета в узле

def split(data, values, probs, column_index, t):
    
    left = np.where(data[:, column_index] <= t)
    right = np.where(data[:, column_index] > t)
        
    true_data = data[left]
    false_data = data[right]
    
    true_values = values[left]
    false_values = values[right]
    
    true_probs = probs[left]
    false_probs = probs[right]
        
    return true_data, false_data, true_values, false_values, true_probs, false_probs



# Нахождение наилучшего разбиения

def find_best_split(data, values, probs, min_samples_leaf):

    # В начальном узле, который будем делить, считаем дисперсию
    root_var = np.var(values)

    best_gain = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique(data[:, index])
        
        for t in t_values:
            true_data, false_data, true_values, false_values, true_probs, false_probs = split(data, values, probs, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_samples_leaf or len(false_data) < min_samples_leaf:
                continue
            
            current_gain = gain(true_values, false_values, root_var)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index

    return best_gain, best_t, best_index



# Построение дерева с помощью рекурсивной функции

def build_tree(data, values, probs, max_depth=3, min_samples_leaf=5):

    gain, t, index = find_best_split(data, values, probs, min_samples_leaf)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if (gain == 0) or (max_depth == 0):
        return Leaf(data, values, probs)

    true_data, false_data, true_values, false_values, true_probs, false_probs = split(data, values, probs, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_values, true_probs, max_depth-1, min_samples_leaf)
    false_branch = build_tree(false_data, false_values, false_probs, max_depth-1, min_samples_leaf)
    
    
    # Возвращаем класс узла со всеми поддеревьями
    return Node(index, t, true_branch, false_branch)

def predict_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        return node.prediction

    if obj[node.index] <= node.t:
        return predict_object(obj, node.true_branch)
    else:
        return predict_object(obj, node.false_branch)
    
def predict(data, tree):
    
    predictions = []
    for obj in data:
        prediction = predict_object(obj, tree)
        predictions.append(prediction)
    return predictions

In [109]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

X, y = make_classification(n_samples=10000, n_features=4, n_informative=4, n_classes=2, n_redundant=0, 
                           n_clusters_per_class=1, random_state=42)

In [110]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = GradientBoostingClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred_train = model.predict_proba(X_train)
y_pred = model.predict_proba(X_test)
print('Train - ', roc_auc_score(y_train, y_pred_train[:, 1]))
print('Test - ', roc_auc_score(y_test, y_pred[:, 1]))

Train -  0.9886394511349922
Test -  0.982067377547222


In [111]:
hand_model = gb_fit(n_trees=100, max_depth=3, X_train=X_train, y_train=y_train, eta=0.1)

In [112]:
target = np.array([np.log(y_train[y_train == 1].shape[0] / 
                               y_train[y_train != 1].shape[0])] * y_train.shape[0]) + gb_predict(X_train, hand_model, eta=0.1)
probs = np.exp(target) / (1 + np.exp(target))
roc_auc_score(y_train, probs)

0.988409690666531

In [113]:
target = np.array([np.log(y_test[y_test == 1].shape[0] / 
                               y_test[y_test != 1].shape[0])] * y_test.shape[0]) + gb_predict(X_test, hand_model, eta=0.1)
probs = np.exp(target) / (1 + np.exp(target))
roc_auc_score(y_test, probs)

0.9820183444008148